In [1]:
# Standard Libraries
import os
import json
import datetime
import numpy as np
import tensorflow as tf

import time
# import nvtx
import torch
from torchsummary import summary

import argparse
import functools
import math
import tf2onnx
# from tensorflow.keras.utils import Progbar

# ------------------------------- CUSTOM FUNCTIONS ------------------------------------------------
# Custom Library
import sys
sys.path.append('../')
    
from proxy_apps.apps.timeseries_prediction import hyperparameters, deepDMD, proxyDeepDMD, proxyDeepDMDMGPU, proxyDeepDMDPyTorch

from proxy_apps.utils.tf import TimingCallback
from proxy_apps.utils.data.main import NpEncoder
from proxy_apps.utils import file_reader, path_handler
from proxy_apps.utils.data.grid import GridNetworkDataHandler, GridNetworkTFDataHandler, GridNetworkNewGen, TransientDataset, GridDataGenPyTorch

In [2]:
print("TensorFlow Version: ", tf.__version__)
print("PyTorch Version: ", torch.__version__)

TensorFlow Version:  2.4.0
PyTorch Version:  1.9.0


In [3]:
import logging

In [4]:
logger = tf.get_logger()

In [5]:
# System Setup
config = file_reader.read_config()

_N_EPOCHS = 2
_BATCH_SIZE = 65536
_APP_NAME = config["info"]["app_name"]
_NROWS = int(config["data"]["n_rows"])
_NCOLS = int(config["data"]["n_cols"])
_REPEAT_COLS = 1 # int(config["data"]["repeat_cols"])
_WINDOW_SIZE = int(config["data"]["window_size"])
_SHIFT_SIZE = int(config["data"]["shift_size"])
_STRIDE = int(config["data"]["stride"])
_N_SIGNALS = int(config["data"]["n_signals"])

_DTYPE = "float32" # config["model"]["dtype"]

_LABEL = "TFDataOpt"
_SUFFIX =  "gpu" + '_' + \
            "a100" + '_' + \
            'ng' + str(1) + '_' + \
            'nc' + str(-1) + '_' + \
            'e' + str(_N_EPOCHS) + '_' + \
            'b' + str(_BATCH_SIZE) + '_' + \
            'r' + str(_REPEAT_COLS) + '_' + _LABEL

performance_dict = dict()

tic = time.time()

# current directory
curr_dir = "./"

# output directory
output_dir = path_handler.get_absolute_path(curr_dir, config["info"]["output_dir"] + config["info"]["name"] + "/" + config["info"]["app_name"] + "/" + _DTYPE + "/R" + str(_REPEAT_COLS) + "/")
if not os.path.exists(output_dir): os.makedirs(output_dir)

# TensorFlow Setup
# logger.info(" [Training Script]: Tensorflow version: %s", tf.__version__)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     log.info("Name:", gpu.name, "  Type:", gpu.device_type)

hyper_param_dict = config["model"]["hyperparameters"]
hyper_param_dict['original_dim']       = _REPEAT_COLS * _NCOLS   # input data dimension
hyper_param_dict['num_epochs']         = _N_EPOCHS  # Number of epochs  
hyper_param_dict['batch_size']         = _BATCH_SIZE

hyper_param_dict['dtype']         = _DTYPE
hp = hyperparameters.HyperParameters(hyper_param_dict)
hp.model_name         = _LABEL

performance_dict["n_epochs"] = hp.ep
performance_dict["batch_size"] = hp.bs

KeyError: 'output_dir'

In [ ]:
# tf.compat.v1.disable_eager_execution()
logger.info("Eager mode: %s", tf.executing_eagerly()) # For easy reset of notebook state.

tf.keras.backend.clear_session()
tf.keras.backend.set_floatx(_DTYPE)

In [ ]:
def get_indexer(n_rows, window_size, shift_size, start_point, leave_last):
    return np.arange(window_size)[None, :] + start_point + shift_size*np.arange(((n_rows - window_size - leave_last - start_point) // shift_size) + 1)[:, None]

In [9]:
# ------------------------------- DATA LOADING ------------------------------------------------   
l_start = time.time()
if _LABEL == "Baseline":
    data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()
elif _LABEL == "TFDataOptPrev":
    data_handler = GridNetworkTFDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()
elif _LABEL == "TFDataOpt":
    data_handler = GridNetworkNewGen(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            d_type=_DTYPE
                                         )

    x_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 0, _N_SIGNALS)
    y_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 1, 0)

    scenario_data = data_handler.get_training_data(x_indexer, y_indexer)
elif _LABEL == "PyTorch":
    scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"])
    dir_list = [scenario_dir + "/" + f + "/" for f in os.listdir(scenario_dir)]
    x_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 0, _N_SIGNALS)
    y_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 1, 0)

    dataset = GridDataGenPyTorch(dir_list, _NROWS, _NCOLS, _REPEAT_COLS, x_indexer, y_indexer)
    train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=_BATCH_SIZE)

l_stop = time.time()
logger.info(' [Training Script]: Time taken for loading datasets: %f seconds', l_stop - l_start)

INFO:tensorflow: [Training Script]: Time taken for loading datasets: 0.258485 seconds


### TensorFlow Implementation

In [10]:
data_options = tf.data.Options()
data_options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
training_dataset = scenario_data.with_options(data_options).batch(hp.bs)

training_dataset = training_dataset.cache()
shuffle_buffer_size = x_indexer.shape[0]*x_indexer.shape[1]*len(data_handler.dir_list) // _BATCH_SIZE
training_dataset = training_dataset.shuffle(buffer_size=x_indexer.shape[0]*x_indexer.shape[1]*len(data_handler.dir_list)//_BATCH_SIZE)
training_dataset = training_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [11]:
# ------------------------------- MODEL TRAINING ------------------------------------------------
model = proxyDeepDMDMGPU.Encoder(hp)
optimizer = tf.optimizers.Adagrad(hp.lr)

# compile and fit model
model.compile(optimizer=optimizer)

trainer = proxyDeepDMDMGPU.NeuralNetworkModel(hp, model)

In [12]:
# timing callback
timing_cb = TimingCallback()

In [13]:
model.build(input_shape=(None, _REPEAT_COLS * _NCOLS))
model.summary()

Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (DenseLayer)     multiple                  17536     
_________________________________________________________________
dense_layer_1 (DenseLayer)   multiple                  16512     
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_layer_2 (DenseLayer)   multiple                  8256      
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_layer_3 (DenseLayer)   multiple                  4160      
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0   

In [14]:
# all_loss = []
# epoch_time = []
# avg_batch_time = []
# for epoch in range(hp.ep):
#     epoch_start_time = time.time()
    
#     # progress bar
#     print("\nEpoch {}/{}".format(epoch+1, hp.ep))
#     pb_i = Progbar(shuffle_buffer_size+1, stateful_metrics=['loss'])
    
#     # metrics
#     total_loss = 0.0
#     num_batches = 0
#     print(total_loss, num_batches)

#     # Iterate over the batches of the dataset.
#     for step, inp_data in enumerate(training_dataset):
        
#         # x and y
#         X, Y        = inp_data
#         # tf.print(X, Y)
#         # print(tf.shape(X), tf.shape(Y))
        
#         with tf.GradientTape() as tape:
#             Psi_X    = model(X, training=True)
#             Psi_Y    = model(Y, training=False)    

#             PSI_X    = tf.concat([X, tf.cast(Psi_X, _DTYPE)], 1)
#             PSI_Y    = tf.concat([Y, tf.cast(Psi_Y, _DTYPE)], 1) 

#             # 1-time step evolution on observable space:
#             K_PSI_X  = tf.matmul(PSI_X, model.KO) 
            
#             # 1-step Koopman loss on observable space:        
#             K_loss   = tf.norm(PSI_Y - K_PSI_X, axis = [0,1], ord = 'fro')

#             # Regularization loss on Koopman operator:
#             Reg_loss= tf.math.scalar_mul(hp.rf, tf.norm(model.KO, axis = [0,1], ord = 'fro'))      
        
#             # Total loss:
#             loss = K_loss + Reg_loss
            
#             # tf.print("K Loss: ", K_loss, "Reg Loss: ", Reg_loss, "Total Loss: ", loss)
#             # loss += sum(self.encoder.losses)
            
#         # Compute gradients
#         trainable_vars = model.trainable_variables
#         gradients = tape.gradient(loss, trainable_vars)
        
#         # Update weights
#         model.optimizer.apply_gradients(zip(gradients, trainable_vars))

#         # Return a dict mapping metric names to current value.
#         # Note that it will include the loss (tracked in self.metrics).
#         total_loss += loss
#         num_batches += 1
        
#         pb_i.add(num_batches//num_batches, values=[('loss', total_loss/num_batches)])

#     # loss_value = total_loss / num_batches
#     epoch_stop_time = time.time()
    
#     print("\n", epoch_stop_time-epoch_start_time)

# #     # logger.info("Loss value: {}".format(tf.keras.backend.get_value(loss_value)))
# #     all_loss.append(loss_value.numpy())
# #     epoch_time.append(epoch_stop_time-epoch_start_time)
# #     avg_batch_time.append((epoch_stop_time-epoch_start_time)/num_batches)

### ONNX Conversion 

In [15]:
model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, 
                                                                  input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float64),
                                                                                   tf.TensorSpec(shape=[None], dtype=tf.float64)])

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [16]:
import onnx
onnx.save(model_proto, 'test.onnx')

In [17]:
# m_start = time.time()
# trainer.fit(training_dataset, n_epochs=_N_EPOCHS, batch_size=_BATCH_SIZE, steps_per_epoch=22)
# m_stop = time.time()
    
# # print info
# logger.info('[INFO]: Time taken for model training (time module): %f seconds', m_stop - m_start)
# logger.info('[INFO]: Time taken for model training (Keras): %f seconds', sum(timing_cb.logs))

### PyTorch Conversion

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [23]:
scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"])
dir_list = [scenario_dir + "/" + f + "/" for f in os.listdir(scenario_dir)]
x_indexer = tuple(get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 0, _N_SIGNALS))
y_indexer = tuple(get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 1, 0))

dataset = GridDataGenPyTorch(dir_list, _NROWS, _NCOLS, _REPEAT_COLS, x_indexer, y_indexer)
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=_BATCH_SIZE)

IndexError: too many indices for array: array is 2-dimensional, but 60 were indexed

In [22]:
x_indexer

array([[   0,    1,    2, ...,  797,  798,  799],
       [  10,   11,   12, ...,  807,  808,  809],
       [  20,   21,   22, ...,  817,  818,  819],
       ...,
       [ 570,  571,  572, ..., 1367, 1368, 1369],
       [ 580,  581,  582, ..., 1377, 1378, 1379],
       [ 590,  591,  592, ..., 1387, 1388, 1389]])

In [24]:
x_indexer

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [26]:
x = np.arange(12).reshape(3,4)
x

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [35]:
indexer = np.array([[0], [1]])
print(tuple(indexer))
x[tuple(indexer)]

(array([0]), array([1]))


array([1])

In [36]:
x[indexer]

array([[[0, 1, 2, 3]],

       [[4, 5, 6, 7]]])

In [20]:
train_features, train_labels = next(iter(train_dataloader))
train_features.shape

torch.Size([65536, 136])

In [21]:
# class Encoder(torch.nn.Module):
#     def __init__(self, hps):
#         super(Encoder, self).__init__()
#         self.KO = torch.nn.Parameter(torch.Tensor(hps.ld+hps.od, hps.ld+hps.od), requires_grad=True)
#         torch.nn.init.normal_(self.KO, mean=0.0, std=0.05)
        
#         self.input_layer   = DenseLayer(hps.h1, hps.od, 0.0, 0.0, hps.d_type)
#         self.hidden_layer1 = DenseLayer(hps.h2, hps.h1, hps.wr, hps.br, hps.d_type)
#         self.dropout_laye1 = torch.nn.Dropout(hps.dr)
#         self.hidden_layer2 = DenseLayer(hps.h3, hps.h2, hps.wr, hps.br, hps.d_type)        
#         self.dropout_laye2 = torch.nn.Dropout(hps.dr)
#         self.hidden_layer3 = DenseLayer(hps.h4, hps.h3, hps.wr, hps.br, hps.d_type)
#         self.dropout_laye3 = torch.nn.Dropout(hps.dr)           
#         self.output_layer  = LinearLayer(hps.ld, hps.h4, hps.wr, hps.br, hps.d_type)
        
#     def forward(self, input_data, training):
#         fx = self.input_layer(input_data)        
#         fx = self.hidden_layer1(fx)
#         if training: fx = self.dropout_laye1(fx)     
#         fx = self.hidden_layer2(fx)
#         if training: fx = self.dropout_laye2(fx) 
#         fx = self.hidden_layer3(fx)
#         if training: fx = self.dropout_laye3(fx) 
#         return self.output_layer(fx)

# class LinearLayer(torch.nn.Module):

#     def __init__(self, units, input_dim, weights_regularizer, bias_regularizer, d_type):
#         super(LinearLayer, self).__init__()
#         self.w = torch.nn.Parameter(torch.Tensor(input_dim, units))
#         torch.nn.init.uniform_(self.w, 
#                                a=-math.sqrt(6/(input_dim+units)),
#                                b=math.sqrt(6/(input_dim+units))
#                               )
        
#         self.b = torch.nn.Parameter(torch.Tensor(units))
#         torch.nn.init.zeros_(self.b)

#     def forward(self, inputs):
#         return torch.add(torch.matmul(inputs, self.w), self.b)

# class DenseLayer(torch.nn.Module):

#     def __init__(self, units, input_dim, weights_regularizer, bias_regularizer, d_type):
#         super(DenseLayer, self).__init__()
#         self.w = torch.nn.Parameter(torch.Tensor(input_dim, units))
#         torch.nn.init.uniform_(self.w, 
#                                a=-math.sqrt(6/(input_dim+units)),
#                                b=math.sqrt(6/(input_dim+units))
#                               )
        
#         self.b = torch.nn.Parameter(torch.Tensor(units))
#         torch.nn.init.zeros_(self.b)

#     def forward(self, inputs):
#         x = torch.add(torch.matmul(inputs, self.w), self.b)
#         return torch.nn.functional.elu(x)

In [22]:
torch.manual_seed(2843)
model = proxyDeepDMDPyTorch.Encoder(hp)
print(model)

Encoder(
  (input_layer): DenseLayer()
  (hidden_layer1): DenseLayer()
  (dropout_laye1): Dropout(p=0.005, inplace=False)
  (hidden_layer2): DenseLayer()
  (dropout_laye2): Dropout(p=0.005, inplace=False)
  (hidden_layer3): DenseLayer()
  (dropout_laye3): Dropout(p=0.005, inplace=False)
  (output_layer): LinearLayer()
)


In [23]:
print(summary(model))

Layer (type:depth-idx)                   Param #
├─DenseLayer: 1-1                        17,536
├─DenseLayer: 1-2                        16,512
├─Dropout: 1-3                           --
├─DenseLayer: 1-4                        8,256
├─Dropout: 1-5                           --
├─DenseLayer: 1-6                        4,160
├─Dropout: 1-7                           --
├─LinearLayer: 1-8                       4,160
Total params: 50,624
Trainable params: 50,624
Non-trainable params: 0
Layer (type:depth-idx)                   Param #
├─DenseLayer: 1-1                        17,536
├─DenseLayer: 1-2                        16,512
├─Dropout: 1-3                           --
├─DenseLayer: 1-4                        8,256
├─Dropout: 1-5                           --
├─DenseLayer: 1-6                        4,160
├─Dropout: 1-7                           --
├─LinearLayer: 1-8                       4,160
Total params: 50,624
Trainable params: 50,624
Non-trainable params: 0


In [24]:
# for name, p in model.named_parameters():
#     print(name, p.shape)

In [25]:
# ko = model.KO
# w0, b0 = model.input_layer.parameters()
# w1, b1 = model.hidden_layer1.parameters()
# w2, b2 = model.hidden_layer2.parameters()
# w3, b3 = model.hidden_layer3.parameters()
# w4, b4 = model.output_layer.parameters()
# optimizer = torch.optim.Adagrad([
#     {'params': [ko, w0, b0, w4]},
#     {'params': w1, 'weight_decay': hp.wr}, {'params': b1, 'weight_decay': hp.br},
#     {'params': w2, 'weight_decay': hp.wr}, {'params': b2, 'weight_decay': hp.br},
#     {'params': w3, 'weight_decay': hp.wr}, {'params': b3, 'weight_decay': hp.br},
#     {'params': b4, 'weight_decay': hp.br}
# ], lr=hp.lr)

In [26]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=hp.lr)
trainer = proxyDeepDMDPyTorch.NeuralNetworkModel(hp, model, optimizer, device)
all_loss = trainer.fit(train_dataloader)

	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
Outside: input size torch.Size([65536, 64]) output_size torch.Size([65536, 64])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
Outside: input size torch.Size([65536, 64]) output_size torch.Size([65536, 64])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
Outside: input size torch.Size([65536, 64]) output_size torch.Size([65536, 64])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
	In Model: input size torch.Size([65536, 136]) output size torch.Size([65536, 128])
Outside: input size torch.Size([65536, 64]) output_size torch.Size([65536, 64])
	In Mode

In [27]:
# for name, p in model.named_parameters():
#     print(name, p)

In [28]:
yp_indexer = get_indexer(_NROWS, _NROWS-1, _SHIFT_SIZE, 0, 1)
yf_indexer = get_indexer(_NROWS, _NROWS-1, _SHIFT_SIZE, 1, 0)
test_size = yp_indexer.shape[0] * yp_indexer.shape[1] * len(data_handler.dir_list)

test_dataset = GridDataGenPyTorch(dir_list, _NROWS, _NCOLS, _REPEAT_COLS, yp_indexer, yf_indexer)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=test_size)

In [29]:
yf_indexer

array([[   1,    2,    3, ..., 1397, 1398, 1399]])

In [30]:
count = 0
for x, y in test_dataset:
    count += 1
print(count)

41970


In [31]:
for x, y in test_dataloader:
    print(x.shape, y.shape)

torch.Size([41970, 136]) torch.Size([41970, 136])


In [32]:
test_size

41970

In [36]:
Psi_X, PSI_X, Psi_Y, PSI_Y, Kloss = trainer.predict(test_dataloader)

	In Model: input size torch.Size([41970, 136]) output size torch.Size([41970, 128])
	In Model: input size torch.Size([41970, 136]) output size torch.Size([41970, 128])
Outside: input size torch.Size([41970, 64]) output_size torch.Size([41970, 64])


In [43]:
print("Koopman loss: %.4f" %Kloss)

print('Psi_X shape:', Psi_X.shape)
print('Psi_Y shape:', Psi_Y.shape)
print('PSI_X shape:', PSI_X.shape)
print('PSI_X shape:', PSI_Y.shape)

# if _LABEL in ["Baseline", "TFDataOptMGPU"]:
#     K_deepDMD = K_model.KO.numpy()
# elif _LABEL in ["TFDataGen"]:
#     K_deepDMD = K_model.encoder.KO.numpy()
# elif _LABEL in ["PyTorch"]:
if torch.cuda.device_count() > 1: K_deepDMD = trainer.encoder.module.KO.data
else: K_deepDMD = trainer.encoder.KO.data
    
print('[INFO]: Shape of Koopman operator', K_deepDMD.shape)
print('[INFO]: Norm of Koopman operator', np.linalg.norm(K_deepDMD))
print('[INFO]: Trace of K_deepDMD:',np.trace(K_deepDMD))
print('[INFO]: One time-step error with K_deepDMD:', np.linalg.norm(PSI_Y - np.matmul(PSI_X, K_deepDMD), ord = 'fro'))

[eigenvaluesK, eigenvectorsK] = np.linalg.eig(K_deepDMD)

Koopman loss: 415.0311
Psi_X shape: torch.Size([41970, 64])
Psi_Y shape: torch.Size([41970, 64])
PSI_X shape: torch.Size([41970, 200])
PSI_X shape: torch.Size([41970, 200])
[INFO]: Shape of Koopman operator torch.Size([200, 200])
[INFO]: Norm of Koopman operator 10.023178
[INFO]: Trace of K_deepDMD: 1.7327344
[INFO]: One time-step error with K_deepDMD: 414.97168
